In [1]:
import os
import sys
sys.path.append(os.path.dirname(os.getcwd()))
import time
import copy
import h5py
import numpy as np
import tensorflow as tf
from subprocess import call
from scipy.misc import imread, imresize
from tf_cnnvis import get_visualization, image_normalization

In [2]:
os.system("python -m wget -o ./alexnet_weights.h5 http://files.heuritech.com/weights/alexnet_weights.h5")

512

In [4]:
# All images in './sample_images" directory
def load_images(output_shape = (224, 224), path = "./sample_images/"):
    filenames = os.listdir(path)
    im = np.zeros((len(filenames), output_shape[0], output_shape[1], 3))

    i = 0
    for file in filenames:
        im[i] = image_normalization(imresize(imread(os.path.join(path, file)), output_shape)) / 255.0
        i += 1
            
    return im, filenames

In [5]:
# loading parameters and mean image for pretrained alexnet model
mean = np.load("./img_mean.npy").transpose((1, 2, 0)) # load mean image of imagenet dataset

f = h5py.File('./alexnet_weights.h5','r')

conv_1 = [f["conv_1"]["conv_1_W"], f["conv_1"]["conv_1_b"]]
conv_2_1 = [f["conv_2_1"]["conv_2_1_W"], f["conv_2_1"]["conv_2_1_b"]]
conv_2_2 = [f["conv_2_2"]["conv_2_2_W"], f["conv_2_2"]["conv_2_2_b"]]
conv_3 = [f["conv_3"]["conv_3_W"], f["conv_3"]["conv_3_b"]]
conv_4_1 = [f["conv_4_1"]["conv_4_1_W"], f["conv_4_1"]["conv_4_1_b"]]
conv_4_2 = [f["conv_4_2"]["conv_4_2_W"], f["conv_4_2"]["conv_4_2_b"]]
conv_5_1 = [f["conv_5_1"]["conv_5_1_W"], f["conv_5_1"]["conv_5_1_b"]]
conv_5_2 = [f["conv_5_2"]["conv_5_2_W"], f["conv_5_2"]["conv_5_2_b"]]
fc_6 = [f["dense_1"]["dense_1_W"], f["dense_1"]["dense_1_b"]]
fc_7 = [f["dense_2"]["dense_2_W"], f["dense_2"]["dense_2_b"]]
fc_8 = [f["dense_3"]["dense_3_W"], f["dense_3"]["dense_3_b"]]

IOError: Unable to open file (Truncated file: eof = 30261256, sblock->base_addr = 0, stored_eoa = 243904576)

In [ ]:
# tensorflow model implementation (Alexnet convolution)
X = tf.placeholder(tf.float32, shape = [None, 224, 224, 3]) # placeholder for input images
y_ = tf.placeholder(tf.float32, shape = [None, 1000]) # placeholder for true labels for input images

radius = 5; alpha = 1e-4; beta = 0.75; bias = 2.0 # hyper parametes for lrn


# Layer - 1 conv1
W_conv_1 = tf.Variable(np.transpose(conv_1[0], (2, 3, 1, 0)))
b_conv_1 = tf.Variable(np.reshape(conv_1[1], (96, )))

y_conv_1 = tf.nn.conv2d(X, filter=W_conv_1, strides=[1, 4, 4, 1], padding="SAME") + b_conv_1
h_conv_1 = tf.nn.relu(y_conv_1, name = "conv1")
h_conv_1 = tf.nn.local_response_normalization(h_conv_1, depth_radius=radius, alpha=alpha, beta=beta, bias=bias)
h_pool_1 = tf.nn.max_pool(h_conv_1, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding="VALID")

h_pool_1_1, h_pool_1_2 = tf.split(axis = 3, value = h_pool_1, num_or_size_splits = 2)



# Layer - 2 conv2
W_conv_2_1 = tf.Variable(np.transpose(conv_2_1[0], (2, 3, 1, 0)))
b_conv_2_1 = tf.Variable(np.reshape(conv_2_1[1], (128, )))

y_conv_2_1 = tf.nn.conv2d(h_pool_1_1, filter=W_conv_2_1, strides=[1, 1, 1, 1], padding="SAME") + b_conv_2_1
h_conv_2_1 = tf.nn.relu(y_conv_2_1, name = "conv2_1")
h_conv_2_1 = tf.nn.local_response_normalization(h_conv_2_1, depth_radius=radius, alpha=alpha, beta=beta, bias=bias)
h_pool_2_1 = tf.nn.max_pool(h_conv_2_1, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding="VALID")


W_conv_2_2 = tf.Variable(np.transpose(conv_2_2[0], (2, 3, 1, 0)))
b_conv_2_2 = tf.Variable(np.reshape(conv_2_2[1], (128, )))

y_conv_2_2 = tf.nn.conv2d(h_pool_1_2, filter=W_conv_2_2, strides=[1, 1, 1, 1], padding="SAME") + b_conv_2_2
h_conv_2_2 = tf.nn.relu(y_conv_2_2, name = "conv2_2")
h_conv_2_2 = tf.nn.local_response_normalization(h_conv_2_2, depth_radius=radius, alpha=alpha, beta=beta, bias=bias)
h_pool_2_2 = tf.nn.max_pool(h_conv_2_2, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding="VALID")

h_pool_2 = tf.concat(axis = 3, values = [h_pool_2_1, h_pool_2_2])



# Layer - 3 conv3
W_conv_3 = tf.Variable(np.transpose(conv_3[0], (2, 3, 1, 0)))
b_conv_3 = tf.Variable(np.reshape(conv_3[1], (384, )))

y_conv_3 = tf.nn.conv2d(h_pool_2, filter=W_conv_3, strides=[1, 1, 1, 1], padding="SAME") + b_conv_3
h_conv_3 = tf.nn.relu(y_conv_3, name = "conv3")

h_conv_3_1, h_conv_3_2 = tf.split(axis = 3, value = h_conv_3, num_or_size_splits = 2)


# h_conv_4_1 = h_conv_3_1
# h_conv_4_2 = h_conv_3_2
# Layer - 4 conv4
W_conv_4_1 = tf.Variable(np.transpose(conv_4_1[0], (2, 3, 1, 0)))
b_conv_4_1 = tf.Variable(np.reshape(conv_4_1[1], (192, )))

y_conv_4_1 = tf.nn.conv2d(h_conv_3_1, filter=W_conv_4_1, strides=[1, 1, 1, 1], padding="SAME") + b_conv_4_1
h_conv_4_1 = tf.nn.relu(y_conv_4_1, name = "conv4_1")


W_conv_4_2 = tf.Variable(np.transpose(conv_4_2[0], (2, 3, 1, 0)))
b_conv_4_2 = tf.Variable(np.reshape(conv_4_2[1], (192, )))

y_conv_4_2 = tf.nn.conv2d(h_conv_3_2, filter=W_conv_4_2, strides=[1, 1, 1, 1], padding="SAME") + b_conv_4_2
h_conv_4_2 = tf.nn.relu(y_conv_4_2, name = "conv4_2")

h_conv_4 = tf.concat(axis = 3, values = [h_conv_4_1, h_conv_4_2])



# Layer - 5 conv5
W_conv_5_1 = tf.Variable(np.transpose(conv_5_1[0], (2, 3, 1, 0)))
b_conv_5_1 = tf.Variable(np.reshape(conv_5_1[1], (128, )))

y_conv_5_1 = tf.nn.conv2d(h_conv_4_1, filter=W_conv_5_1, strides=[1, 1, 1, 1], padding="SAME") + b_conv_2_1
h_conv_5_1 = tf.nn.relu(y_conv_5_1, name = "conv5_1")
h_conv_5_1 = tf.nn.local_response_normalization(h_conv_5_1, depth_radius=radius, alpha=alpha, beta=beta, bias=bias)
h_pool_5_1 = tf.nn.max_pool(h_conv_5_1, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding="VALID")


W_conv_5_2 = tf.Variable(np.transpose(conv_5_2[0], (2, 3, 1, 0)))
b_conv_5_2 = tf.Variable(np.reshape(conv_5_2[1], (128, )))

y_conv_5_2 = tf.nn.conv2d(h_conv_4_2, filter=W_conv_5_2, strides=[1, 1, 1, 1], padding="SAME") + b_conv_2_2
h_conv_5_2 = tf.nn.relu(y_conv_5_2, name = "conv5_2")
h_conv_5_2 = tf.nn.local_response_normalization(h_conv_5_2, depth_radius=radius, alpha=alpha, beta=beta, bias=bias)
h_pool_5_2 = tf.nn.max_pool(h_conv_5_2, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding="VALID")

h_pool_5 = tf.concat(axis = 3, values = [h_pool_5_1, h_pool_5_2])

dimensions = h_pool_5.get_shape().as_list()
dim = dimensions[1] * dimensions[2] * dimensions[3]

# # Part of Alexnet model which is not required for deconvolution
h_flatten = tf.reshape(h_pool_5, shape=[-1, dim])

# Layer - 6 fc6
W_full_6 = tf.Variable(np.array(fc_6[0]))
b_full_6 = tf.Variable(np.array(fc_6[1]))

y_full_6 = tf.add(tf.matmul(h_flatten, W_full_6), b_full_6)
h_full_6 = tf.nn.relu(y_full_6, name = "fc6")
h_dropout_6 = tf.nn.dropout(h_full_6, 0.5)


# Layer - 7 fc7
W_full_7 = tf.Variable(np.array(fc_7[0]))
b_full_7 = tf.Variable(np.array(fc_7[1]))

y_full_7 = tf.add(tf.matmul(h_dropout_6, W_full_7), b_full_7)
h_full_7 = tf.nn.relu(y_full_7, name = "fc7")
h_dropout_7 = tf.nn.dropout(h_full_7, 0.5)


# Layer - 8 fc8
W_full_8 = tf.Variable(np.array(fc_8[0]))
b_full_8 = tf.Variable(np.array(fc_8[1]))

y_full_8 = tf.add(tf.matmul(h_dropout_7, W_full_8), b_full_8, name = "fc8")

In [ ]:
# im, filenames = load_images(path = "./sample_images")
im = np.expand_dims(image_normalization(imresize(imread(os.path.join("./sample_images", "images.jpg")), (224, 224))) / 255.0, axis = 0)

In [ ]:
layers = ["conv1", "conv2_1", "conv2_2", "p"]
total_time = 0

start = time.time()
get_visualization(graph_or_path = tf.get_default_graph(), value_feed_dict = {X : im}, layers=layers)
start = time.time() - start
print("Total Time = %f" % (start))